In [1]:
import pandas as pd
#Import SpaCy
import spacy
import regex
from spacy import displacy
from collections import Counter
import copy
from fuzzywuzzy import process
#target_nlp = spacy.load('de_core_news_sm')
IN = "input/dev.roen.df.short.tsv"
OUT = "output/masked-levenshtein-1/dev.roen.masked.tsv"
JSON = "output/masked-levenshtein-1/dev.roen.mapping.json"
entities = ["PER", "PERSON", "ORG", "ORGANIZATION", "PRODUCT", "WORK_OF_ART", "LOC"]

In [10]:
def mask(input_csv, output_csv, output_json, entities, lang):
    
#Set up dict & dataframe
    d = {}
    df = input_csv

    if lang == "roen":
        source_nlp = spacy.load("ro_core_news_sm")
    else:
        source_nlp = spacy.load("en_core_web_sm")

    entity_count = 0
    for index, row in df.iterrows():
        line1, line2 = row.original, row.translation
        source = source_nlp(line1)
        source_adjust, target_adjust = 0, 0
        
#mask source:
        for X in source.ents:
            if X.label_ in entities:
                if X.text not in d:
                    d[X.text] = "NE{}".format(entity_count)
                start, end = X.start_char-source_adjust, X.end_char-source_adjust
                line1 = "".join([line1[:start],d[X.text],line1[end:]])
                source_adjust=len(X.text)-len(d[X.text])
                entity_count+=1
  
#Find NE from source in target and mask:
                entity = "".join(["(?b)" + "(", X.text, ")", "{0<=s<=5,0<=d<=5}"])
                re = regex.search(entity, line2)
                if re:
                    d[X.text] = "NE{}".format(entity_count)
                    t_start, t_end = re.span()[0]-target_adjust, re.span()[1]-target_adjust
                    line2 = "".join([line2[0:re.span()[0]], d[X.text], line2[re.span()[1]:]])
                    target_adjust=(re.span()[1]-re.span()[0])-len(d[X.text])
                    entity_count+=1


#Replace row with masked version:
        df.loc[index,"original"]=line1
        df.loc[index,"translation"]=line2
#Export dict to json:
    return df 

# EN-DE
This is a notebook detaiolign the masking of the en de dataset at different levels

## dev data

In [ ]:
en_de_dev_df =  pd.read_csv("../data/en-de/dev.ende.df.short.tsv",sep="\t")

result_dev = pd.read_csv("../masked-data/dev_en_de.tsv", sep="\t")

result_dev

In [ ]:
result_dev["error"] = abs(result_dev["labels"]-result_dev["predictions"])
result_dev

In [ ]:
result_dev = result_dev.sort_values(by=["error"],ascending=False)
result_dev = result_dev.head(400)
result_dev.reset_index(inplace=True,drop=True)
result_dev

In [ ]:
maskd_df = pd.DataFrame()
for index, text in result_dev["text_a"].items():
    try:
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == text].tolist()[0]
        s = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the dev data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(text,en_de_dev_df["original"].values.tolist())[0]
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == matched_string].tolist()[0]
        wanted_row = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,wanted_row], ignore_index=True)

In [ ]:
maskd_df = mask(maskd_df,OUT,JSON,entities,"ende")

In [ ]:
total_dev_df= pd.concat([maskd_df,en_de_dev_df], ignore_index=True)

total_dev_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_dev_df.to_csv("../masked-data/masked_dev_en_de_40.tsv", index = None, header=True,sep = '\t')

total_dev_df

# Training Data

In [66]:
en_de_train_df =  pd.read_csv("../data/en-de/train.ende.df.short.tsv",sep="\t")


result_train = pd.read_csv("../masked-data/train_en_de.tsv", sep="\t")

result_train["labels"] = en_de_train_df["z_mean"]

result_train["error"] = abs(result_train["labels"]-result_train["predictions"])

In [67]:
result_train

,index,text_a,text_b,predictions,labels,error
0,0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,-0.254870,1.119409,1.374278
1,1,"However, a disappointing ninth in China meant ...",Eine enttäuschende Neunte in China bedeutete j...,-0.368015,-0.488591,0.120575
2,2,"In his diary, Chase wrote that the release of ...","In seinem Tagebuch, Chase schrieb, dass die Ve...",-0.362235,-2.207007,1.844771
3,3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,-0.467312,-0.799946,0.332635
4,4,Once North Pacific salmon die off after spawni...,Sobald der nordpazifische Lachs nach dem Laich...,-0.386398,0.381633,0.768031
...,...,...,...,...,...,...
6995,6995,Some may also discourage or disallow unsanitar...,Einige können auch unhygienische Praktiken wie...,-0.324950,0.544875,0.869824
6996,6996,"In the late 1860s, the crinolines disappeared ...",In den späten 1860er Jahren verschwanden die K...,-0.332789,0.271088,0.603877
6997,6997,"Disco was criticized as mindless, consumerist,...","Disco wurde als geistlos, konsumistisch, überp...",-0.334347,-1.104549,0.770203
6998,6998,Planters would then fill large hogsheads with ...,Die Pflanzer würden dann große Heuschrecken mi...,-0.361690,-0.207402,0.154289


In [68]:
result_train = result_train.sort_values(by=["error"],ascending=False)
num = int(7000 * 0.4)
result_train = result_train.head(num)

result_train.reset_index(inplace=True,drop=True)
result_train

,index,text_a,text_b,predictions,labels,error
0,6975,He vacated the WWA Cruiserweight title in Apri...,Nach seiner Rückkehr zum WWF verließ er im Apr...,-0.308271,-7.541931,7.233660
1,2080,"Haroun and S.A. Mourad, Proceedings of Interna...","Haroun and S.A. Mourad, Proceedings of Interna...",-0.361857,-7.194496,6.832639
2,6229,Boone Trails Grand Towers Gravois Trail New Ho...,Boone Trails Grand Towers Gravois Trail New Ho...,-0.500930,-7.043685,6.542755
3,2500,"""Radio Birdman"", in The Trouser Press Record G...","""Radio Birdman"", in The Hose Press Record Guid...",-0.308249,-6.847839,6.539590
4,2214,Illustrated Encyclopedia of Woodworking Handto...,Illustrated Encyclopedia of Woodworking Handto...,-0.302977,-6.832649,6.529672
...,...,...,...,...,...,...
2795,3841,"Therese joined this confraternity on April 26,...",Therese schloss sich dieser Bruderschaft am 26...,-0.244245,0.426869,0.671115
2796,4217,The Julia Tuttle Causeway connects Midtown and...,Der Julia Tuttle Causeway verbindet Midtown mi...,-0.297415,0.373602,0.671017
2797,5117,"A little later, however, in the foreign conces...",Wenig später jedoch in der ausländischen Konze...,-0.307805,0.363141,0.670945
2798,3589,"Jan Hus, a theologian and rector at the Charle...","Jan Hus, Theologe und Rektor der Karls-Univers...",-0.261028,-0.931644,0.670616


In [69]:

maskd_df = pd.DataFrame()
for index, text in result_train["text_a"].items():
    try:
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == text].tolist()[0]
        s = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the train data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(text,en_de_train_df["original"].values.tolist())[0]
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == matched_string].tolist()[0]
        wanted_row = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,wanted_row], ignore_index=True)

In [ ]:
maskd_df = mask(maskd_df,OUT,JSON,entities,"ende")

In [ ]:
total_train_df = pd.concat([maskd_df,en_de_train_df], ignore_index=True)
total_train_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_train_df.to_csv("../masked-data/masked_train_en_de_40.tsv", index = None, header=True,sep = '\t')

total_train_df

In [59]:
total_train_df.to_csv("../masked-data/masked_train_en_de_20.tsv", index = None, header=True,sep = '\t')